# Mobile Robot Control

### Implementation of Robot Behaviors


#### Wall-following Behavior
This function implements a wall-following behavior with constant linear speed.


In [9]:
def follow_wall_to_left(kd, d_l, d_desired):
    """ Follows the wall to the left of the robot.
    Input Parameters: 
        kd = controller gain;
        d_desired = desired robot distance to the wall;
        d_l = measured distance to the left wall;
    Returns:
        u_ref = reference linear speed command;
        w_ref = reference angluar speed command.
    """
    u_ref = 0.5     # [m/s] some constant linear speed
    w_ref = kd*(d_l - d_desired)
    return u_ref, w_ref

Assume you want to follow the wall at 15cm, and the robot is actually at a distance of 18cm. When the function is called, it will return the desired speed values that the robot should have to get closer to the wall.


In [10]:
d_desired = 0.15    # [m]
d_l = 0.18          # [m] - this value must be calculated based on sensor readings

# Controller gain - define how the reaction of the robot will be:
# higher controller gain will result in more faster reaction, but it can cause oscillations
kd = 1

u_ref, w_ref = follow_wall_to_left(kd, d_l, d_desired)
print(f"Desired linear speed  = {u_ref}m/s")
print(f"Desired angular speed = {w_ref}rad/s")

Desired linear speed  = 0.5m/s
Desired angular speed = 0.03rad/s


#### Wall-following Behavior with Obstacle Detection
This function implements a wall-following behavior with linear speed that depends on the distance to obstacle.


In [11]:
def follow_wall_to_left_obst(kd, d_l, d_desired, d):
    """ Follows the wall to the left of the robot.
    Input Parameters: 
        kd = controller gain;
        d_desired = desired robot distance to the wall;
        d_l = measured distance to the left wall;
        d = measured distance to the obstacle (front sensor);
    Returns:
        u_ref = reference linear speed command;
        w_ref = reference angluar speed command.
    """
    # Variables:
    d_min = 0.05     # [m] minimum admissible distance to the obstacle; 
    d_max = 0.25     # [m] maximum measurable distance by the front sensor;
    u_max = 0.5      # [m/s] linear speed for d = d_max

    u_ref = u_max*d/d_max if d >= d_min else 0
    w_ref = kd*(d_l - d_desired)
    return u_ref, w_ref

Assume you want to follow the wall at 15cm, and the robot is actually at a distance of 18cm. When the function is called, it will return the desired speed values that the robot should have to get closer to the wall.

Test the function with different values of d.


In [12]:
d = 0.5
d_desired = 0.15    # [m]
d_l = 0.18          # [m] - this value must be calculated based on sensor readings

# Controller gain - define how the reaction of the robot will be:
# higher controller gain will result in more faster reaction, but it can cause oscillations
kd = 1

u_ref, w_ref = follow_wall_to_left_obst(kd, d_l, d_desired, d)
print(f"Desired linear speed  = {u_ref}m/s")
print(f"Desired angular speed = {w_ref}rad/s")

Desired linear speed  = 1.0m/s
Desired angular speed = 0.03rad/s


The above implementations of wall-following behavior will likely cause the robot to perform poorly. To make sure that the robot will follow the wall, it is a good idea to include another term that tries to kwwp the robot parallel to the wall. For that, we assume that the robot has two sensors that measure its distance to the left wall, one in the front and another in the back of the robot. The sensors are placed on a distance _s_ from each other.

In [13]:
def improved_follow_wall_to_left_obst(kd, kd2, d_fl, d_rl, d_desired, d):
    """ Follows the wall to the left of the robot.
    Input Parameters: 
        kd = controller gain for controlling the distance to the wall;
        kd2 = controller gain for keeping the robot parallel to the wall;
        d_desired = desired robot distance to the wall;
        d_fl = distance to the left wall measured by the front sensor;
        d_rl = distance to the left wall measured by the rear sensor;
        d = measured distance to the obstacle (front sensor);
    Returns:
        u_ref = reference linear speed command;
        w_ref = reference angluar speed command.
    """
    # Variables:
    d_min = 0.05     # [m] minimum admissible distance to the obstacle; 
    d_max = 0.25     # [m] maximum measurable distance by the front sensor;
    u_max = 0.5      # [m/s] linear speed for d = d_max

    u_ref = u_max * d/d_max if d >= d_min else 0
    d_l = (d_fl + d_rl)/2
    w_ref = kd*(d_l - d_desired) + kd2*(d_fl - d_rl)
    return u_ref, w_ref

Test it with different values for _d_fl_ and _d_rl_.

In [14]:
d = 0.5
d_desired = 0.15    # [m]
d_fl = 0.18         # [m]
d_rl = 0.20         # [m]


# Controller gain - define how the reaction of the robot will be:
# higher controller gain will result in more faster reaction, but it can cause oscillations
kd = 1
kd2 = 1

u_ref, w_ref = improved_follow_wall_to_left_obst(kd, kd2, d_fl, d_rl, d_desired, d)
print(f"Desired linear speed  = {u_ref}m/s")
print(f"Desired angular speed = {w_ref}rad/s")

Desired linear speed  = 1.0m/s
Desired angular speed = 0.01999999999999999rad/s
